In [47]:
import numpy as np
import pandas as pd
import csv
import datetime
from datapathfile import path_lookup_dir, datafile
import re

In [48]:
datafile

'rtb_data.csv'

In [49]:
def listcolumn(x):
    x = x.translate(None, '"')
    x = x.translate(None, ' ')
    x = x[1:-1].split(",")
    j = x[-1]
    return j

In [50]:
def TrueFalse(value):
    if value[0]=='F':
        value = False
    else:
        value = True
    return value

In [51]:
def Model(device):
    if device[0]=='F':
        value = False
    else:
        value = True
    return value

In [52]:
filename='ATA_data.csv'

In [53]:
reader = list(csv.reader(open(path_lookup_dir+datafile, 'rb'), delimiter='\t'))
colheaders = reader[0] 
clean_reader = [r[0:20] for r in reader]
df = pd.DataFrame(clean_reader)
df.columns = [colheaders]
df.drop(0,axis=0, inplace= True)

In [54]:
len(df['advertiser_bundle_id'].unique())

1

In [55]:
df.drop(['advertiser_bundle_id','creative_id','app_id','bidadop_id','device_idfa'],1, inplace= True)

In [56]:
def device_filter(text):
    text = str(text)
    if 'iPhone' in text:
        return 'Phone'
    elif 'iPod'in text:
        return 'Music Player'
    else:
        return 'Tablet'

In [57]:
df['device'] = pd.Series(df['model'].map(device_filter))

In [58]:
df['impression'] = df['impression'].apply(TrueFalse)
df['installed'] = df['installed'].apply(TrueFalse)
df['clicked'] = df['clicked'].apply(TrueFalse)

In [59]:
df.ix[:, 'datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")
df.ix[:, 'app_type'] = df['app_categories'].apply(listcolumn)


In [60]:
drop_index=df[np.logical_and(
        np.logical_and(df['impression'], df['installed']),
        ~ df['clicked'])].index

In [61]:
empty_index = np.where(df['app_type'] == '')
ul_index = np.where(df['app_type'] == 'ul')
df = df.drop(df.index[empty_index])
df = df.drop(df.index[ul_index])
df = df.drop(df.index[drop_index])

In [62]:
df['weekday'] = df['datetime'].map(lambda x: x.isoweekday())
df['hours'] = df['datetime'].map(lambda x: x.hour)
df.ix[:, 'week'] = df['datetime'].map(lambda x:x.isocalendar()[1])

In [63]:
df['osv'] = df['osv'].map(lambda x: int(x[0]))

In [64]:
df.columns

Index([u'datetime', u'ad_id', u'app_name', u'app_categories',
       u'publisher_name', u'os', u'osv', u'make', u'model', u'country',
       u'city', u'language', u'impression', u'clicked', u'installed',
       u'device', u'app_type', u'weekday', u'hours', u'week'],
      dtype='object')

In [65]:
impressiondf = df.groupby('city')['impression','clicked'].sum()
impression_index = impressiondf[impressiondf['impression'] > 5].index

In [66]:
df.to_csv(filename)

In [ ]:
filename